### Data Loading

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Example: Load a CSV file
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/ISYE 6740 Project Files/data_cleaned.csv/part-00000-0db4eea5-232e-4109-a268-f9e84543421d-c000.csv') # Replace 'your_file.csv' with your file path
df.head()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Prscrbr_NPI,Prscrbr_Last_Org_Name,Prscrbr_First_Name,Prscrbr_State_Abrvtn,Prscrbr_Type,Long_acting_opioid,Total_Claims,Total_30day_Fills,Total_Day_Supply,Total_Drug_Cost,Covered_Recipient_NPI,Total_Payment_Sum,Num_Payments_Sum,Is_Physician_Owned,Is_Charity,Is_Entity,Is_Individual,Is_Covered,Fraud
0,1003000126,Enkeshafi,Ardalan,MD,Internal Medicine,NaN,169,296.8,8388,11872.51,1003000126,21.52,1.0,0,0,0,0,1,0
1,1003000167,Escobar,Julio,NV,Dentist,NaN,65,65.0,348,197.10,1003000167,88.63,3.0,0,0,0,0,1,0
2,1003000423,Velotta,Jennifer,OH,Obstetrics & Gynecology,NaN,80,172.9,4962,12430.06,1003000423,449.20,21.0,0,0,0,0,1,0
3,1003000597,Kim,Dae,OK,Urology,0.0,4116,5471.0,92160,220361.52,1003000597,1857.08,70.0,0,0,0,0,1,0
4,1003000639,Benharash,Peyman,CA,Cardiac Surgery,NaN,14,14.0,420,143.36,1003000639,16755.15,56.0,0,0,0,0,1,0


In [ ]:
# Merge into 1
df['Fraud'].value_counts()

,count
Fraud,
0,569033
1,133


In [ ]:
df['Long_acting_opioid'] = df['Long_acting_opioid'].fillna(0)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569166 entries, 0 to 569165
Data columns (total 19 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Prscrbr_NPI            569166 non-null  int64  
 1   Prscrbr_Last_Org_Name  569166 non-null  object 
 2   Prscrbr_First_Name     569165 non-null  object 
 3   Prscrbr_State_Abrvtn   569166 non-null  object 
 4   Prscrbr_Type           569166 non-null  object 
 5   Long_acting_opioid     569166 non-null  float64
 6   Total_Claims           569166 non-null  int64  
 7   Total_30day_Fills      569166 non-null  float64
 8   Total_Day_Supply       569166 non-null  int64  
 9   Total_Drug_Cost        569166 non-null  float64
 10  Covered_Recipient_NPI  569166 non-null  int64  
 11  Total_Payment_Sum      569166 non-null  float64
 12  Num_Payments_Sum       569166 non-null  float64
 13  Is_Physician_Owned     569166 non-null  int64  
 14  Is_Charity             569166 non-nu

In [ ]:
df.columns

Index(['Prscrbr_NPI', 'Prscrbr_Last_Org_Name', 'Prscrbr_First_Name',
       'Prscrbr_State_Abrvtn', 'Prscrbr_Type', 'Long_acting_opioid',
       'Total_Claims', 'Total_30day_Fills', 'Total_Day_Supply',
       'Total_Drug_Cost', 'Covered_Recipient_NPI', 'Total_Payment_Sum',
       'Num_Payments_Sum', 'Is_Physician_Owned', 'Is_Charity', 'Is_Entity',
       'Is_Individual', 'Is_Covered', 'Fraud'],
      dtype='object')

### Process Data

In [ ]:
from re import X
# X = df.drop(['Fraud', 'Prscrbr_NPI', 'Prscrbr_Last_Org_Name', 'Prscrbr_First_Name','Covered_Recipient_NPI'], axis=1)
X = df.drop(['Fraud', 'Prscrbr_NPI', 'Prscrbr_Last_Org_Name', 'Prscrbr_First_Name','Covered_Recipient_NPI','Is_Charity', 'Is_Entity', 'Is_Individual',
       'Is_Covered', 'Prscrbr_State_Abrvtn'], axis=1)
X = pd.get_dummies(X)
y = df['Fraud']

In [ ]:
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler
Scaler = StandardScaler().fit(X)
X_scaled = Scaler.transform(X)

Split Train Test

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
#Scaled
X_train_scaled, X_test_scaled, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

0 only data for Novelty Detection.

In [ ]:
# Scale Data
X_train_scaled_0 = X_train_scaled[y_train == 0]
X_test_scaled_0 = X_test_scaled[y_test == 0]

Create smaller version testing and training data

In [ ]:
# Randomly select 14235 of the X_train_scaled and X_train corresponding to y_train = 0 for easier model training...
import random
import numpy as np
random.seed(42)

train_indices = random.sample(range(len(X_train_scaled_0)), 14235)

# Combine with the X_train corresponding to y_train = 1 for a smaller training set...
X_train_scaled_1 = X_train_scaled[y_train == 1]
X_train_1 = X_train[y_train == 1]
X_train_scaled_small = np.concatenate([X_train_scaled_0[train_indices], X_train_scaled_1])
X_train_small = pd.concat([X_train.iloc[train_indices], X_train_1])

# get the y_train corresponding to the smaller set...
y_train_1 = y_train[y_train == 1]
y_train_small = pd.concat([y_train.iloc[train_indices], y_train_1])

In [ ]:
y_train_small.value_counts()

,count
Fraud,
0,14232
1,109


In [ ]:
test_indices = random.sample(range(len(X_test_scaled)), 4745)

# Repeat same process for testing data
X_test_scaled_1 = X_test_scaled[y_test == 1]
X_test_1 = X_test[y_test == 1]
X_test_scaled_small = np.concatenate([X_test_scaled_0[test_indices], X_test_scaled_1])
X_test_small = pd.concat([X_test.iloc[test_indices], X_test_1])

y_test_1 = y_test[y_test == 1]
y_test_small = pd.concat([y_test.iloc[test_indices], y_test_1])

In [ ]:
y_test_small.value_counts()

,count
Fraud,
0,4742
1,30


In [ ]:
from imblearn.over_sampling import RandomOverSampler, SMOTE
X_ros, y_ros = RandomOverSampler(random_state=42).fit_resample(X_train_scaled_small, y_train_small)
X_smote, y_smote = SMOTE(random_state=42).fit_resample(X_train_scaled_small, y_train_small)

### Test Outlier Detection

#### LOF

In [ ]:
from sklearn.neighbors import LocalOutlierFactor
from sklearn.metrics import classification_report
import pandas as pd

# Your samplings dictionary
samplings = {
    "Original": (X_train_scaled_small, y_train_small),
    "Random Oversampling": (X_ros, y_ros),
    "SMOTE": (X_smote, y_smote)
}

# List to store results
results = []

for name, (X_train, y_train) in samplings.items():
    # Fit LOF only on the normal class (0)
    lof = LocalOutlierFactor(novelty=True, n_neighbors=13, contamination=0.02)
    lof.fit(X_train[y_train == 0])

    # Predict on the test set
    lof_pred = lof.predict(X_train_scaled_small)

    # Convert predictions: outliers (-1) → 1, inliers (1) → 0
    lof_pred = (lof_pred == -1).astype(int)

    # Classification report for class 1 (novelties)
    report = classification_report(y_train_small, lof_pred, output_dict=True, zero_division=0)['1']

    # Save metrics
    results.append({
        "Model": "LOF (Novelty)",
        "Sampling": name,
        "Precision": report["precision"],
        "Recall": report["recall"],
        "F1 Score": report["f1-score"]
    })

# Create and display results DataFrame
lof_nov_results_df = pd.DataFrame(results)
print(lof_nov_results_df)


           Model             Sampling  Precision    Recall  F1 Score
0  LOF (Novelty)             Original   0.023622  0.055046  0.033058
1  LOF (Novelty)  Random Oversampling   0.023622  0.055046  0.033058
2  LOF (Novelty)                SMOTE   0.023622  0.055046  0.033058


In [ ]:
from sklearn.neighbors import LocalOutlierFactor
from sklearn.metrics import classification_report
from tqdm import tqdm_notebook
precision = []
recall = []
f1_score = []

for i in tqdm_notebook(range(50)):
  lof = LocalOutlierFactor(n_neighbors=i+1, contamination=0.005)
  lof_pred = lof.fit_predict(X_train_scaled_small)
  lof_pred = (lof_pred == -1).astype(int)
  lof_report = classification_report(y_train_small, lof_pred, output_dict=True, zero_division=0)["1"]

  precision.append(lof_report["precision"])
  recall.append(lof_report["recall"])
  f1_score.append(lof_report["f1-score"])

print('Best n_neighbors (precision): ', precision.index(max(precision)))
print('Best n_neighbors (recall): ', recall.index(max(recall)))
print('Best n_neighbors (f1-score): ', f1_score.index(max(f1_score)))

<ipython-input-123-2acaa5614d8d>:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(50)):


  0%|          | 0/50 [00:00<?, ?it/s]

Best n_neighbors (precision):  12
Best n_neighbors (recall):  12
Best n_neighbors (f1-score):  12


In [ ]:
# Your samplings dictionary
samplings = {
    "Original": (X_train_scaled_small, y_train_small),
    "Random Oversampling": (X_ros, y_ros),
    "SMOTE": (X_smote, y_smote)
}

# List to store results
results = []

for name, (X_train, y_train) in samplings.items():
    # Fit LOF only on the normal class (0)
    lof = LocalOutlierFactor(n_neighbors=13, contamination=0.01)
    lof_pred = lof.fit_predict(X_train_scaled_small)
    lof_pred = (lof_pred == -1).astype(int)

    # Classification report for class 1 (novelties)
    report = classification_report(y_train_small, lof_pred, output_dict=True, zero_division=0)['1']

    # Save metrics
    results.append({
        "Model": "LOF (Outiers)",
        "Sampling": name,
        "Precision": report["precision"],
        "Recall": report["recall"],
        "F1 Score": report["f1-score"]
    })

# Create and display results DataFrame
lof_results_df = pd.DataFrame(results)
print(lof_results_df)



           Model             Sampling  Precision    Recall  F1 Score
0  LOF (Outiers)             Original   0.034722  0.045872  0.039526
1  LOF (Outiers)  Random Oversampling   0.034722  0.045872  0.039526
2  LOF (Outiers)                SMOTE   0.034722  0.045872  0.039526


In [ ]:
from imblearn.combine import SMOTETomek, SMOTEENN
from imblearn.over_sampling import ADASYN
from collections import Counter

# SMOTE + Tomek Links (removes borderline/ambiguous points)
smote_tomek = SMOTETomek(random_state=42)
X_smote_tomek, y_smote_tomek = smote_tomek.fit_resample(X_train_scaled_small, y_train_small)
print("After SMOTETomek:", Counter(y_smote_tomek))

# SMOTE + ENN (removes misclassified samples based on neighbors)
smote_enn = SMOTEENN(random_state=42)
X_smote_enn, y_smote_enn = smote_enn.fit_resample(X_train_scaled_small, y_train_small)
print("After SMOTEENN:", Counter(y_smote_enn))

# ADASYN (focuses on harder examples in the minority class)
adasyn = ADASYN(random_state=42, n_neighbors=5)
X_adasyn, y_adasyn = adasyn.fit_resample(X_train_scaled_small, y_train_small)
print("After ADASYN:", Counter(y_adasyn))

After SMOTETomek: Counter({0: 14170, 1: 14170})
After SMOTEENN: Counter({1: 13704, 0: 12693})
After ADASYN: Counter({0: 14232, 1: 14206})


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import pandas as pd

# Your samplings dictionary
samplings = {
    "Original": (X_train_scaled_small, y_train_small),
    "Random Oversampling": (X_ros, y_ros),
    "SMOTE": (X_smote, y_smote),
    "SMOTE + Tomek Links": (X_smote_tomek, y_smote_tomek),
    "SMOTE + ENN": (X_smote_enn, y_smote_enn),
    "Adasyn": (X_adasyn, y_adasyn)
}

# List to collect results
results = []

# Define the classifier with your hyperparameters
rf = RandomForestClassifier(
    n_estimators=200,
    max_depth=60,
    min_samples_split=2,
    min_samples_leaf=1,
    random_state=42
)

for name, (X_train, y_train) in samplings.items():
    # Fit the classifier
    rf.fit(X_train, y_train)

    # Predict on the test set
    rf_pred = rf.predict(X_test_scaled_small)

    # Evaluate metrics for class '1'
    report = classification_report(y_test_small, rf_pred, output_dict=True, zero_division=0)['1']

    # Store the results
    results.append({
        "Model": "Random Forest",
        "Sampling": name,
        "Precision": report["precision"],
        "Recall": report["recall"],
        "F1 Score": report["f1-score"]
    })

# Display the results as a DataFrame
rf_results_df = pd.DataFrame(results)
print(rf_results_df)

           Model             Sampling  Precision  Recall  F1 Score
0  Random Forest             Original        0.0     0.0       0.0
1  Random Forest  Random Oversampling        0.0     0.0       0.0
2  Random Forest                SMOTE        0.0     0.0       0.0
3  Random Forest  SMOTE + Tomek Links        0.0     0.0       0.0
4  Random Forest          SMOTE + ENN        0.0     0.0       0.0
5  Random Forest               Adasyn        0.0     0.0       0.0


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import pandas as pd

# Your samplings dictionary
samplings = {
    "Original": (X_train_scaled_small, y_train_small),
    "Random Oversampling": (X_ros, y_ros),
    "SMOTE": (X_smote, y_smote),
    "SMOTE + Tomek Links": (X_smote_tomek, y_smote_tomek),
    "SMOTE + ENN": (X_smote_enn, y_smote_enn),
    "Adasyn": (X_adasyn, y_adasyn)
}

results = []

# Loop through each sampling strategy
for name, (X_train, y_train) in samplings.items():
    # Define logistic regression model
    weights = {0: 1, 1: 10}
    lr = LogisticRegression(random_state=42, class_weight=weights)  # 'liblinear' is good for small datasets or binary classification

    # Fit model
    lr.fit(X_train, y_train)

    # Predict on the test set
    y_pred = lr.predict(X_test_scaled_small)

    # Classification report for class '1'
    report = classification_report(y_test_small, y_pred, output_dict=True, zero_division=0)['1']

    # Store metrics
    results.append({
        "Model": "Logistic Regression",
        "Sampling": name,
        "Precision": report["precision"],
        "Recall": report["recall"],
        "F1 Score": report["f1-score"]
    })

# Convert to DataFrame
lgresults_df = pd.DataFrame(results)
print(lgresults_df)


                 Model             Sampling  Precision  Recall  F1 Score
0  Logistic Regression             Original   0.000000     0.0  0.000000
1  Logistic Regression  Random Oversampling   0.006820     0.9  0.013537
2  Logistic Regression                SMOTE   0.007181     0.9  0.014248
3  Logistic Regression  SMOTE + Tomek Links   0.007183     0.9  0.014252
4  Logistic Regression          SMOTE + ENN   0.007264     0.9  0.014412
5  Logistic Regression               Adasyn   0.007158     0.9  0.014203


In [ ]:
from tensorflow.keras.metrics import Precision, Recall
import tensorflow.keras.backend as K
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Activation, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report
# Your samplings dictionary
samplings = {
    "Original": (X_train_scaled_small, y_train_small),
    "Random Oversampling": (X_ros, y_ros),
    "SMOTE": (X_smote, y_smote),
    "SMOTE + Tomek Links": (X_smote_tomek, y_smote_tomek),
    "SMOTE + ENN": (X_smote_enn, y_smote_enn),
    "Adasyn": (X_adasyn, y_adasyn)
}

results = []

# Loop through each sampling strategy
for name, (X_train, y_train) in samplings.items():
    # Define a new model each time to avoid weight carryover

    model = Sequential()

    model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
    # Input layer
    model.add(Dense(32, name='dense_1'))
    model.add(BatchNormalization(name='batch_norm_1'))
    model.add(Activation('relu', name='activation_1'))
    model.add(Dropout(0.5, name='dropout_1'))

    # Hidden layer
    model.add(Dense(32, name='dense_2'))
    model.add(BatchNormalization(name='batch_norm_2'))
    model.add(Activation('relu', name='activation_2'))
    model.add(Dropout(0.5, name='dropout_2'))

    # More Hidden layers
    model.add(Dense(32, name='dense_3'))
    model.add(BatchNormalization(name='batch_norm_3'))
    model.add(Activation('relu', name='activation_3'))
    model.add(Dropout(0.5, name='dropout_3'))

    # Output layer
    model.add(Dense(1, name='dense_4'))
    model.add(Activation('sigmoid', name='activation_4'))


    # Compile model
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='binary_crossentropy',
                  metrics=[Precision(), Recall()])

    # Train model
    model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=0)

    # Predict on test data
    y_pred_probs = model.predict(X_test_scaled_small)
    y_pred = (y_pred_probs >= 0.5).astype(int).flatten()

    # Classification report for class '1'
    report = classification_report(y_test_small, y_pred, output_dict=True, zero_division=0)['1']

    # Store metrics
    results.append({
        "Model": "Neural Network",
        "Sampling": name,
        "Precision": report["precision"],
        "Recall": report["recall"],
        "F1 Score": report["f1-score"]
    })

# Convert to DataFrame
nn_results_df = pd.DataFrame(results)
print(nn_results_df)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
            Model             Sampling  Precision    Recall  F1 Score
0  Neural Network             Original   0.000000  0.000000  0.000000
1  Neural Network  Random Oversampling   0.016453  0.300000  0.031196
2  Neural Network                SMOTE   0.006761  0.333333  0.013254
3  Neural Network  SMOTE + Tomek Links   0.007488  0.366667  0.014676
4  Neural Network          SMOTE + ENN   0.012953  0.333333  0.024938
5  Neural Network               Adasyn   0.013436  0.233333  0.025408


In [ ]:
X.columns

Index(['Long_acting_opioid', 'Total_Claims', 'Total_30day_Fills',
       'Total_Day_Supply', 'Total_Drug_Cost', 'Total_Payment_Sum',
       'Num_Payments_Sum', 'Is_Physician_Owned', 'Prscrbr_Type_Acupuncturist',
       'Prscrbr_Type_Addiction Medicine',
       ...
       'Prscrbr_Type_Substance Abuse Rehabilitation Facility',
       'Prscrbr_Type_Surgery', 'Prscrbr_Type_Surgical Oncology',
       'Prscrbr_Type_Thoracic Surgery',
       'Prscrbr_Type_Thoracic Surgery (Cardiothoracic Vascular Surgery)',
       'Prscrbr_Type_Undefined Physician type',
       'Prscrbr_Type_Undersea and Hyperbaric Medicine',
       'Prscrbr_Type_Unknown Supplier/Provider Specialty',
       'Prscrbr_Type_Urology', 'Prscrbr_Type_Vascular Surgery'],
      dtype='object', length=148)